In [5]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

dataset = np.loadtxt("./features.txt", delimiter=" ");
print(dataset.shape)

X = dataset[:, :34]
Y = dataset[:, 35]
botnetDataset = dataset[dataset[:,35]==1, :]
normalDataset = dataset[dataset[:,35]==0, :]

botnetX = botnetDataset[:,:34]
botnetY = botnetDataset[:, 35]

normalX = normalDataset[:,:34]
normalY = normalDataset[:, 35]

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledBotnetX = scaler.fit_transform(botnetX)
rescaledNormalX = scaler.fit_transform(normalX)
rescaledX = scaler.fit_transform(X)


(52580, 36)


In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(rescaledX, Y)

model.score(rescaledX, Y)

0.92700646633701023

In [7]:
def GetNextBotnetSet(rescaledX, Y):
    step = 4000
    prevStep = 0
    while step < rescaledX.shape[0]:
        yield rescaledX[prevStep:step, :], Y[prevStep:step]
        prevStep = step
        step = step + 4000
        
    return rescaledX[prevStep:, :], Y[prevStep:]

In [8]:
from sklearn.cross_validation import train_test_split

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model2 = LogisticRegression()
    model2 = model2.fit(X_train, Y_train)
    print(model2.score(X_test, Y_test))

0.688463911166
0.806909315238
0.837754472548
0.727945712523
0.873534855028
0.872917951882
0.874151758174
0.873534855028
0.87476866132
0.869216533004
0.820481184454
0.87476866132


In [9]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    scores = cross_val_score(LogisticRegression(), X, Y, scoring='accuracy', cv=10)
    predicted = cross_val_predict(LogisticRegression(), X, Y, cv=10)
    print(scores.mean(), metrics.accuracy_score(Y, predicted))

0.668945669879 0.668929761758
0.813721057679 0.813726700407
0.848790397467 0.848784100728
0.720174148666 0.720158005185
0.876312889132 0.876311566473
0.87927554764 0.879274163683
0.883596687522 0.883594617948
0.880880029228 0.880878903839
0.881621987791 0.881619553142
0.8797693748 0.879767929885
0.838183008327 0.838168127392
0.877675328432 0.877669423528
